# População por subprefeitura

Para calcularmos diversos dados de maneira proporcional à população por subprefeitura, precisamos inicialmente obter a população de cada uma delas. Para isso, utilizamos os dados obtidos do [TabNet](https://prefeitura.sp.gov.br/web/saude/w/tabnet/30417), já agregados por Subprefeitura. Para a população em geral, utilizaremos os dados de 2022, de acordo com o último censo. Já para a população de 0 a 74 anos de idade, utilizaremos os dados de 2024, que se tratam de uma estimativa realizada pela Fundação SEADE.

In [ ]:
import pandas as pd
from os import environ, path, makedirs
from dotenv import load_dotenv
from unidecode import unidecode

In [ ]:
load_dotenv()

# Obtendo os dados

## Obtendo os dados da população geral

In [ ]:
data_path = path.join('data', 'input')
pop_total_filepath = path.join(data_path, 'A205703192_29_141_165.csv')

df_pop_total = pd.read_csv(
    pop_total_filepath,
    encoding='latin1',
    sep=';',
    skiprows=3,
    dtype=str
)
df_pop_total.rename(columns={'População':'pop_total'}, inplace=True)

df_pop_total

## Obtebdo os dados da população até 74 anos

In [ ]:
pop74_filepath = path.join(data_path,'saude', 'A171453192_29_141_45.csv')

df_pop74 = pd.read_csv(
    pop74_filepath,
    encoding='latin1',
    sep=';',
    skiprows=4,
    dtype=str
)

df_pop74.head()
df_pop74.rename(columns={'População':'pop_0_74'}, inplace=True)
df_pop74

## CSV de Subprefeituras do Qlik

In [ ]:
url_subs = environ.get('CSV_SUBPREFEITURAS_QLIK')
df_subs_qlik = pd.read_csv(url_subs)
df_subs_qlik

In [ ]:
df_subs_qlik = df_subs_qlik[['sub.CODIGO', 'sub.NOME']]
df_subs_qlik

# Transformando os dados

## Removendo linhas indesejadas

A importação também trouxe linhas de totalização e fonte ao final da tabela, então vamos removê-las

In [ ]:
df_pop_total = df_pop_total.iloc[:-2,:].copy()
df_pop_total

In [ ]:
df_pop74 = df_pop74.iloc[:-2,:].copy()
df_pop74

## Padronizando o nome das subprefeituras

Finalmente, vamos padronizar os nomes das subprefeituras para o mesmo padrão do qlik sense.

In [ ]:
df_pop_total['Subprefeitura (Sub)'] = df_pop_total['Subprefeitura (Sub)'].apply(lambda x: unidecode(x.strip().upper()))
df_pop_total

In [ ]:
df_pop74['Subprefeitura (Sub)'] = df_pop74['Subprefeitura (Sub)'].apply(lambda x: unidecode(x.strip().upper()))
df_pop74

In [ ]:
subs_mapper = {
    s: q
    for s, q in zip(df_pop_total['Subprefeitura (Sub)'].sort_values(),
                    df_subs_qlik['sub.NOME'].sort_values())
}

subs_mapper

O dicionário acima mapeia corretamente os nomes de subprefeituras vindos do tabnet com os nomes das subprefeituras na tabela vinda do qlik sense, portanto utilizaremos o dicionário para criar a coluna de nomes padronizados.

In [ ]:
df_pop_total['sub.NOME'] = df_pop_total['Subprefeitura (Sub)'].map(subs_mapper)
df_pop_total

In [ ]:
df_pop74['sub.NOME'] = df_pop74['Subprefeitura (Sub)'].map(subs_mapper)
df_pop74

In [ ]:
df_pop_total.drop(columns=['Subprefeitura (Sub)'], inplace=True)
df_pop74.drop(columns=['Subprefeitura (Sub)'], inplace=True)

In [ ]:
df_pop_total = df_pop_total[['sub.NOME', 'pop_total']].copy()
df_pop_total

In [ ]:
df_pop74 = df_pop74[['sub.NOME', 'pop_0_74']].copy()
df_pop74

# Exportando os arquivos

Neste notebook, vamos apenas salvar os arquivos extraídos na pasta de entrada de dados.

In [ ]:
output_dir = path.join('data', 'input')

if not path.exists(output_dir):
    makedirs(output_dir)


filename=path.join(output_dir, 'subprefeituras-populacao')
df_pop_total.to_csv(f'{filename}.csv',
            sep=';',
            decimal=',',
            encoding='latin1',
            index=False
            )


filename=path.join(output_dir, 'saude', 'subprefeituras-populacao-0-74')
df_pop74.to_csv(f'{filename}.csv',
            sep=';',
            decimal=',',
            encoding='latin1',
            index=False
            )